<a href="https://colab.research.google.com/github/JahnaviRamagiri/EVA-B2/blob/master/Assignment_Final_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)        #input: 28X28X1 OUtput: 26X26X16 RF: 3
        #self.batch1=nn.BatchNorm2d(16)     
        self.drop=nn.Dropout(0.1)               #input -? OUtput? RF
        self.conv2 = nn.Conv2d(16, 32, 3)       #input: 26X26X16 OUtput: 24X24X32 RF: 5
        #self.batch2=nn.BatchNorm2d(32)
        #self.drop=nn.Dropout(0.25)   
        self.pool1 = nn.MaxPool2d(2, 2)         #input: 24X24X32 OUtput: 12X12X32 RF: 10
        self.conv3 = nn.Conv2d(32, 8, 1)        #input: 12X12X32 OUtput: 12X12X8 RF: 10
        self.conv8=nn.Conv2d(8,16,3,padding=1)  #input: 12X12X8 OUtput: 12X12X16 RF: 12
        #self.batch1=nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 32, 3)       #input: 12X12X16 OUtput: 10X10X32 RF: 14
        #self.batch2=nn.BatchNorm2d(32)
       
        self.pool2 = nn.MaxPool2d(2, 2)         #input: 10X10X32 OUtput: 5X5X32 RF: 28
        self.conv5 = nn.Conv2d(32, 16, 1)       #input: 5X5X32 OUtput: 5X5X16 RF: 28
        #self.batch1=nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 32, 3)       #input: 5X5X16 OUtput: 3X3X32 RF: 30
        #self.batch2=nn.BatchNorm2d(32)
        self.conv7 = nn.Conv2d(32, 10, 3)       #input: 3X3X32 OUtput: 1X1X10 RF: 32

    def forward(self, x):
        x = self.pool1(self.drop(F.relu(self.conv2(self.drop(F.relu(self.conv1(x)))))))
        x = self.pool2(self.drop(F.relu(self.conv4(self.drop(F.relu(self.conv8(self.drop(F.relu(self.conv3(x))))))))))
        x = self.drop(F.relu(self.conv6(self.drop(F.relu(self.conv5(x))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [15]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 32, 24, 24]           4,640
           Dropout-4           [-1, 32, 24, 24]               0
         MaxPool2d-5           [-1, 32, 12, 12]               0
            Conv2d-6            [-1, 8, 12, 12]             264
           Dropout-7            [-1, 8, 12, 12]               0
            Conv2d-8           [-1, 16, 12, 12]           1,168
           Dropout-9           [-1, 16, 12, 12]               0
           Conv2d-10           [-1, 32, 10, 10]           4,640
          Dropout-11           [-1, 32, 10, 10]               0
        MaxPool2d-12             [-1, 32, 5, 5]               0
           Conv2d-13             [-1, 16, 5, 5]             528
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nepoch:',epoch,' Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [18]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


epoch: 1  Test set: Average loss: 0.0941, Accuracy: 9735/10000 (97.35%)



  0%|          | 0/938 [00:00<?, ?it/s]


epoch: 2  Test set: Average loss: 0.0494, Accuracy: 9839/10000 (98.39%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


epoch: 17  Test set: Average loss: 0.0199, Accuracy: 9938/10000 (99.38%)



  0%|          | 0/938 [00:00<?, ?it/s]


epoch: 18  Test set: Average loss: 0.0179, Accuracy: 9941/10000 (99.41%)



loss=0.005453705787658691 batch_id=937: 100%|██████████| 938/938 [00:23<00:00, 39.68it/s]



epoch: 19  Test set: Average loss: 0.0182, Accuracy: 9947/10000 (99.47%)

